In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Panel
import csv
import matplotlib.pyplot as plt
from collections import Counter
import sklearn
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import log_loss
import datetime
import random
from collections import defaultdict
from numpy import isnan

### Glogal features

In [2]:
impression = pd.read_pickle('../data/processed/joined_impressions.pkl').fillna(0)

In [3]:
impression['date'] = impression['impressionTimestamp'].dt.date

In [4]:
articles = pd.read_pickle('../data/processed/articles.pkl')

In [5]:
satisfiedlist = []
for i in list(impression.maxDuration):
    if i >= 10000:
        satisfiedlist.append(1)
    else:
        satisfiedlist.append(0)

impression['satisfiedlist'] = satisfiedlist        
    

In [6]:
category_flatten = [val for sublist in articles.categories.values for val in sublist]

category_flatten = list(set(category_flatten))

In [7]:
rich_impression = impression.merge(articles, left_on="contentId",right_on="id")

In [8]:
from sklearn import preprocessing
from itertools import chain

def columns_to_key_feature_pairs(row, key_column, feature_columns):
    return [(row[key_column], '{}={}'.format(column, row[column])) for column in feature_columns]

def array_column_to_key_feature_pairs(row, key_column, array_column):
    return [(row[key_column], u'{}={}'.format(array_column, value)) for value in row[array_column]]

feature_columns = ['id', 'hotness', 'lifetime']

item_features = pd.DataFrame.from_records(
    data=chain.from_iterable(
        columns_to_key_feature_pairs(row, key_column='id', feature_columns=feature_columns) +\
        array_column_to_key_feature_pairs(row, key_column='id', array_column='categories') +\
        array_column_to_key_feature_pairs(row, key_column='id', array_column='tags')
        for _, row in articles.iterrows()), 
    columns=['item', 'feature_name'],
    index='item')

#item_features.head()

In [9]:
max_position = impression.contentPosition.max()
content_positions = np.arange(max_position+1)

context_features = pd.DataFrame.from_dict(
    {'contentPosition': content_positions, 'feature_name': ['position={}'.format(p) for p in content_positions]}
).set_index('contentPosition')

#context_features

In [10]:
all_features = pd.concat([item_features, context_features],axis=0)

all_features.feature_name = all_features.feature_name.astype('category')
all_features['encoded_feature'] = all_features.feature_name.cat.codes

#all_features.head()

### user with category

In [11]:
environmentlist = list(set(rich_impression["environmentId"].values))

In [12]:
grouped = rich_impression.groupby(["environmentId"])
dic = {}
for item in environmentlist:
    l = grouped.get_group(item)["categories"].tolist()
    flatten =  list(set([item for sublist in l for item in sublist]))
    dic[item] = flatten
    
    

In [13]:
userinfor = pd.DataFrame({"environmentid": list(dic.keys()),"category": list(dic.values())})

flat_userinfor = pd.DataFrame(
    data=[(index, category)
         for index, row in userinfor.iterrows()
         for category in row.category],
    columns=['index', 'category']).set_index("index")

### we cluster different users into 20 groups based on interests

In [14]:
from sklearn.preprocessing import LabelEncoder 

from scipy import sparse

import scipy.sparse as sp

def feature_matrix(rich_impressions, column_name):
    
    category_encoder = LabelEncoder()

    rich_impressions = rich_impressions.assign(encoded_column=lambda df: category_encoder.fit_transform(df[column_name]))

    row_indexes = rich_impressions.index.values    
    column_indexes = rich_impressions.encoded_column.values
    
    output = sparse.coo_matrix(
        (np.ones_like(column_indexes), 
        (row_indexes, column_indexes)))    
    
    return output
    

In [15]:
from sklearn.preprocessing import OneHotEncoder

def singletag(mergedate, name):
    
    enc = OneHotEncoder()

    X = enc.fit_transform(mergedate[[name]].values)
    
    #result = X.toarray()
    
    return X
    

In [16]:

from sklearn.cluster import KMeans  

def kmeans(X, n_clusters):

    labeler = KMeans(n_clusters) 

    labeler.fit(X.tocsr())  
    
    return labeler


In [17]:
def indices( mylist, value):
    return [i for i,x in enumerate(mylist) if x==value]


In [18]:
X_user = feature_matrix(flat_userinfor, "category")

labeler = kmeans(X_user, 20)


### Select just one groups to do the experient

In [744]:
index_6 = indices(labeler.labels_,19)

### Find rich_impression  data set for each group

In [745]:
def grouped_rich_impression(indexlist, userinfor, rich_impression):
    idx = userinfor.index.isin(indexlist)
    groupeduserinf = userinfor[idx]
    groupedenvironment = groupeduserinf.environmentid.values.tolist()
    im_idx = rich_impression.environmentId.isin(groupedenvironment)
    group_rich_impression = rich_impression[im_idx]
    return groupeduserinf,group_rich_impression
    

In [746]:
groupeduserinf,grouped_rich_impression = grouped_rich_impression(index_6, userinfor, rich_impression)



### There are seven days, we can seen as seven frontpages

In [747]:
def CTRforday(grouped_rich_impression):

    up = grouped_rich_impression.groupby(["date"]).aggregate({"isClicked":sum})
    
    down = grouped_rich_impression.groupby(["date"]).aggregate({"isClicked":len})
    
    CTR = up.isClicked.values/down.isClicked.values
    
    up["CTR"] = CTR
    
    result = up.sort(columns="CTR")
    
    return result

In [748]:
result = CTRforday(grouped_rich_impression) 

result

/Users/XiHUANG/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,isClicked,CTR
date,,
2017-06-18,322.0,0.029801
2017-06-14,90.0,0.040089
2017-06-17,306.0,0.040121
2017-06-13,79.0,0.042405
2017-06-19,91.0,0.047028
2017-06-16,91.0,0.047669
2017-06-15,80.0,0.049813


### caculate the initial action value for each category

In [749]:
def CTRperCategory(impression,  all_features):
    
    batch_impressions = impression[['frontPageViewId','environmentId','contentPosition', 'contentId', 'isClicked','date','satisfiedlist']]\
        .assign(sample_id=lambda df: np.arange(df.shape[0]))\
        .set_index('sample_id')

    featurelize_impressions = pd.concat([
        #pd.merge(batch_impressions, all_features, left_on='environmentId', right_index=True),
        pd.merge(batch_impressions, all_features, left_on='contentId', right_index=True),
        pd.merge(batch_impressions, all_features, left_on='contentPosition', right_index=True)],
        axis=0).sort_index()
    
    featurelize_impressions = featurelize_impressions[featurelize_impressions.feature_name.str.startswith('cat')]

    featurelize_impressions = featurelize_impressions.fillna(0)
   
    up = featurelize_impressions.groupby(["feature_name"]).aggregate({"isClicked":sum}) 
    
    down = featurelize_impressions.groupby(["feature_name"]).aggregate({"isClicked":len})
    
    CTR = up.isClicked.values/down.isClicked.values
    
    up["CTR"] = CTR
    
    result = up["CTR"].to_dict()
    
   
    
    return featurelize_impressions, result
    

In [750]:
featurelize_impressions,Q_0 = CTRperCategory(grouped_rich_impression,  all_features)

### Caculate the initial CTR value for each content

### CCM

In [751]:
def sat(dataset):
    
    up = featurelize_impressions.groupby(["feature_name"]).aggregate({"satisfiedlist":sum})
    
    down = featurelize_impressions.groupby(["feature_name"]).aggregate({"isClicked":sum})
    
    up["sat"] = up.satisfiedlist.values/down.isClicked.values
    
    result = up.fillna(0)
    
    result = result.reset_index()
    
    result = result.set_index("feature_name")["sat"].to_dict()
    
    return result

In [752]:
sat_prob = sat(grouped_rich_impression)

### tua probability calculate

In [753]:
shrinkdataset = grouped_rich_impression[["frontPageViewId","contentPosition","contentId","satisfiedlist","isClicked"]]

In [754]:


def tua_inf(newtua_impression):
    
    #newtua_impression = grouped_rich_impression[["frontPageViewId","contentPosition","contentId","satisfiedlist","isClicked"]]
    
    #down = len(set(process_dataset.frontPageViewId.values))
    
    idx  = newtua_impression.groupby(['frontPageViewId'])["contentPosition"].transform(max) == newtua_impression["contentPosition"]

    newtua_impression = newtua_impression[idx]
    
    count = Counter(newtua_impression.contentPosition.values)
    
    a = list(count.values())
    
    n = len(a)
    
    examination_inf = {}
    
    for i in range(len(a)):
   
        examination_inf[i+1] = sum(a[i:n])
    
    temp = list( examination_inf.values())
    
    tua_probability = {}
    for i in range(1,62):
         tua_probability[i] = 0
    
    
    
    tua_probability[1] = 1
    
    for i in range(len(a)):
        if i == 0:
            tua_probability[i+1] = 0
        else:
            tua_probability[i+1] = temp[i]/temp[i-1]


    return  tua_probability
    

    

In [755]:
dataset = shrinkdataset[((shrinkdataset['satisfiedlist'] == 0) & (shrinkdataset['isClicked'] == 0))]

tua_s0c0 = tua_inf(dataset) # = tua 1

#tua_s0c0

In [756]:
dataset = shrinkdataset[((shrinkdataset['satisfiedlist'] == 0) & (shrinkdataset['isClicked'] == 1))]

tua_s0c1 = tua_inf(dataset)

#tua_s0c1

In [757]:
dataset = shrinkdataset[((shrinkdataset['satisfiedlist'] == 1) & (shrinkdataset['isClicked'] == 1))]

tua_s1c1 = tua_inf(dataset)

#tua_s1c1

### /epsilon greedy Bandit/Qlearning

In [758]:
parameter_threshold = 0.6
parameter_alpha = 0.15
epsilon = 0
countdic = defaultdict(int)
new_frontpage = pd.DataFrame()
#sat_state = []
#click_state = []

position = 1

def rewardactionfun(Q,sat_prob,examination,epsilon):
    
    if random.random() > epsilon:
        d = dict((k, v) for k, v in Q.items() if v == max(Q.values()))
        action = random.choice(list(d.keys()))
    else:
        action = random.choice(list(Q.keys()))

    reward = sat_prob[action]*examination
    return action,reward

In [759]:
action,reward = rewardactionfun(Q_0,sat_prob,1,epsilon)
countdic[action]+=1

In [760]:


def stateidentify(satvalue,reward):
   
    if(satvalue >= parameter_threshold):
        #sat_state.append(1)
        sat_state = 1
    else:

        #sat_state.append(0)
        sat_state = 0

    if(reward >= parameter_threshold):
        #click_state.append(1)
        click_state = 1
    else:
        #click_state(0)
        click_state = 0
    return click_state,sat_state

click_state,sat_state = stateidentify(sat_prob[action],reward)



In [761]:
new_frontpage= pd.DataFrame({"position":[position], "context":action,"sat":[sat_state],"reward":[reward],"click":[click_state]})

In [762]:
new_frontpage

,click,context,position,reward,sat
0,1,categories= sulten,1,1.0,1


In [763]:
def actionvalueupdate(alpha,action,reward,Q,count):
    copy_Q = Q
    old_Q = copy_Q[action]
    Q[action] = (old_Q + alpha*(reward-old_Q))/count
    Q = {k: Q[k] for k in Q if not isnan(Q[k])}
    
    return Q

In [764]:
Q = actionvalueupdate(parameter_alpha,action,reward,Q_0,countdic[action])

In [765]:
def examinationfunc(position,pre_sat_state,pre_click_state):
    
    if pre_sat_state == 1 and pre_click_state == 1:
        return tua_s1c1[position]
    if pre_sat_state == 0 and pre_click_state == 1:
        return tua_s0c1[position]
    if pre_sat_state == 0 and pre_click_state == 0:
        return tua_s0c0[position]
    if pre_sat_state == 1 and pre_click_state == 0:
        return 0
        
  
    
     
    
  

In [766]:
position=2
while(position <= 61):
    #print(position)
    Q_pre = Q
    #print(Q_pre)
    #print(sat_state)
    #print(click_state)
    examination = examinationfunc(position,sat_state,click_state)
    #print(examination)
    if examination == 0:
        break
    else:
   
        action,reward = rewardactionfun(Q_pre,sat_prob,examination,epsilon)
        #print(action)
        #print(reward)
        countdic[action] += 1
        click_state,sat_state = stateidentify(sat_prob[action],reward)
        #print(click_state)
        #print(sat_state)
        new_frontpage = new_frontpage.append(pd.DataFrame({"position":[position], "context":action,"sat":[sat_state],"reward":[reward],"click":[click_state]}))
        Q = actionvalueupdate(parameter_alpha,action,reward,Q_pre,countdic[action])
        position+=1
       
    

In [767]:
new_frontpage.groupby("context")["context"].count().values.max()

4

### user fatigue

In [768]:
temp = featurelize_impressions.groupby(["environmentId","date","feature_name"]).aggregate({"feature_name":len}).rename(columns = {"feature_name":"frequency"})

In [769]:
temp = temp.reset_index()

In [770]:
temp.groupby(["environmentId","date"])["frequency"].max().values.mean()

5.2305491990846678